In [33]:
import pandas as pd
import re
import snowballstemmer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.metrics import f1_score,accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import nlp_odev

In [16]:
# veri temizlemesi için örnek veri kümemiz okunur.
df_1 = pd.read_csv("data/nlp.csv",index_col=0)
df_1.head()

,Text
0,"Yapay Zeka Nedir, Ne İşe Yarar?\n\nYapay zeka ..."
1,🔎 Verilerinizi analiz edin: \nChatGPT'yi kulla...
2,"Öncelikle yapay zekanın ne olduğunu, #dijitala..."
3,9. Baidu Research: Çin'in önde gelen arama mot...
4,4. Microsoft AI: Microsoft'un yapay zeka tekno...


In [17]:
### tanımlanan df_1 içerisinde Text sütununu ön işlem fonksiyonlarından geçirerek Text_2 olarak df_1 içerisinde yeni bir sütun oluşturun. ###
df_1["Text_2"] = df_1['Text'].apply(nlp_odev.pre_processing)
df_1["Text_2"] = df_1['Text_2'].apply(nlp_odev.remove_space)
df_1.head()

,Text,Text_2
0,"Yapay Zeka Nedir, Ne İşe Yarar?\n\nYapay zeka ...","[yapay, zeka, nedir, işe, yarar, yapay, zeka, ..."
1,🔎 Verilerinizi analiz edin: \nChatGPT'yi kulla...,"[veri, analiz, edin, chatgptyi, kullanarak, do..."
2,"Öncelikle yapay zekanın ne olduğunu, #dijitala...","[öncelik, yapay, zeka, olduğunu, iç, önemli, o..."
3,9. Baidu Research: Çin'in önde gelen arama mot...,"[baidu, research, çin, ön, gele, ara, motor, b..."
4,4. Microsoft AI: Microsoft'un yapay zeka tekno...,"[microsoft, ai, microsoft, yapay, zeka, teknol..."


In [18]:
### df_1 içerisinde Text_2 sütununda boş liste kontrolü ###

df1_index = df_1[df_1["Text_2"].str[0].isnull()].index
df_1 = df_1.drop(df1_index)
df_1 = df_1.reset_index()
del df_1["index"]

df_1[df_1["Text_2"].str[0].isnull()]

,Text,Text_2


In [19]:
### word2vec model oluşturma ###
model = Word2Vec(sentences = df_1["Text_2"].tolist(),vector_size = 100, window = 5,min_count=1)
model.save("data/word2vec.model")

# df_1 dataframe mizi artık kullanmaycağımızdan ram de yer kaplamaması adına boş bir değer ataması yapıyoruz.
df_1 = {}
#############################################################################################################################################

In [20]:
# sınıflandırma yapacağımız veri okunur.
df_2 = pd.read_csv("data/metin_siniflandirma.csv",index_col=0)
df_2.head()

,Text,Label
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın


In [21]:
### tanımlanan df_2 içerisinde Text sütununu ön işlem fonksiyonlarından geçirerek Text_2 olarak df_2 içerisinde yeni bir sütun oluşturun. ###
df_2["Text_2"] = df_2['Text'].apply(nlp_odev.pre_processing)
df_2["Text_2"] = df_2['Text_2'].apply(nlp_odev.remove_space)
df_2.head()

,Text,Label,Text_2
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si..."
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, zama, çalıştı, ür, stok, bit, diyor, mal,..."
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, di, bağıracak, kal]"
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, amk, bu, yıl, çıkar, bu, boku,..."
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize..."


In [22]:
### df_2 içerisinde Text_2 sütununda boş liste kontrolü ###
df_2[df_2["Text_2"].str[0].isnull()]
df2_index = df_2[df_2["Text_2"].str[0].isnull()].index
df_2 = df_2.drop(df2_index)
df_2 = df_2.reset_index()
del df_2["index"]

df_2[df_2["Text_2"].str[0].isnull()]

,Text,Label,Text_2


In [24]:
### sınıflandırma yapacağımız df_2 içerisinde bulunan Text_2 sütun verisini word2vec verisinde güncelleyin. ### 
nlp_odev.word2vec_update(df_2["Text_2"])
df_2

,Text,Label,Text_2
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si..."
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, zama, çalıştı, ür, stok, bit, diyor, mal,..."
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, di, bağıracak, kal]"
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, amk, bu, yıl, çıkar, bu, boku,..."
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize..."
...,...,...,...
3993,Aynı şeyleri yapmaktan sıkıldım,üzgün,"[aynı, yapmak, sıkıl]"
3994,"Olacağı varsa oluyor ya hani, biteceği varsa d...",üzgün,"[olacak, var, oluyor, hani, bitecek, var, bits..."
3995,Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM,üzgün,"[istiyor, am, so, fiyasko, mutsuz]"
3996,Garip bir şekilde canım çok yanıyor moralim bozuk,üzgün,"[garip, şekil, ca, yanıyor, moral, bozuk]"


In [25]:
print(len(df_2["Text_2"]))

3998


In [26]:
### Text_2 sütun üzerinden word2vec adında bu modeli kullanarak yeni bir sütun yaratın
df_2["word2vec"] = df_2["Text_2"].apply(nlp_odev.word2vec_analysis)
df_2.head()

,Text,Label,Text_2,word2vec
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si...","[-0.0015750041930004954, 0.006984548643231392,..."
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, zama, çalıştı, ür, stok, bit, diyor, mal,...","[-0.00024980504531413317, 0.003929830156266689..."
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, di, bağıracak, kal]","[0.0013833744451403618, 0.00909792073071003, -..."
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, amk, bu, yıl, çıkar, bu, boku,...","[-0.007988453842699528, 0.021176952868700027, ..."
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize...","[-0.0036815074272453785, 0.011395451612770557,..."


In [30]:
### word2vec sütunumuzu train test olarak bölün ###
msg_train, msg_test, label_train, label_test = train_test_split(df_2["word2vec"].tolist(),df_2["Label"].tolist(),test_size = 0.2,random_state=42)

In [31]:
### svm pipeline oluştur, modeği eğit ve test et ###
svm = Pipeline([('svm',LinearSVC())])
svm.fit(msg_train, label_train)
y_pred_class = svm.predict(msg_test)

In [34]:
### accuracy ve f1 score çıktısını print ile gösterin. ###
print("svm accuracy score : ", accuracy_score(label_test, y_pred_class))
print("svm f1 score : ", f1_score(label_test, y_pred_class, average="weighted"))

svm accuracy score :  0.38125
svm f1 score :  0.3835890656368794
